In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader

torch.manual_seed(1)
train_X = torch.load("./train_X.pt")
train_Y = torch.load("./train_Y.pt")

train_data = TensorDataset(train_X, train_Y)
train_dataloader = DataLoader(train_data, batch_size=1, shuffle=True)

In [2]:
from torch import nn

# nn.Moduleを継承してモデルを作成
class Perseptron(nn.Module):
  def __init__(self, input_size, output_size) -> None:
    super().__init__()

    # nn.Linear: 全結合層を作成する
    self.fc = nn.Linear(input_size, output_size, bias=False)

    # 重み行列を正規分布に上書き
    nn.init.normal_(self.fc.weight, 0.0, 1.0)

  def forward(self, x):
    return torch.softmax(self.fc(x), dim=-1)
  
model = Perseptron(train_X.size()[1], 4)
model.load_state_dict(torch.load("./knock71.model.pt"))
loss_fn = nn.CrossEntropyLoss()

In [3]:
model.fc.weight

Parameter containing:
tensor([[ 0.8807,  2.5288,  0.8601,  ...,  0.4481,  0.1001,  1.5422],
        [ 0.9785,  1.6087,  0.8545,  ...,  0.0863, -1.5734, -0.6372],
        [-0.6044, -2.1389,  1.1721,  ..., -1.2481, -0.5696, -0.3201],
        [ 0.4554, -1.5279, -1.8458,  ...,  0.8399,  0.3675, -0.1449]],
       requires_grad=True)

In [4]:
from torch import optim

op = optim.SGD(model.parameters(), lr=0.1)

In [5]:
epochs = 100

for i in range(epochs):
  for j, (data, label) in enumerate(train_dataloader):
    pred = model(data)
    loss = loss_fn(pred, label)

    op.zero_grad()
    loss.backward()
    op.step()

model.fc.weight

Parameter containing:
tensor([[-0.3521,  2.4999, -0.9548,  ..., -1.6718,  5.2874, -1.8399],
        [ 1.6600, -0.0357,  3.3315,  ...,  1.6676, -6.4627, -2.1711],
        [ 1.7609, -0.6373, -1.4983,  ..., -2.8058, -1.0960,  0.7305],
        [-1.3579, -1.3540,  0.1603,  ...,  2.9359,  0.6031,  3.7193]],
       requires_grad=True)

In [6]:
torch.save(model.state_dict(), "knock73.model.pt")